# Setup

### additional dependencies/configuration

In [2]:
!pip install geopandas --user -q

### modules

In [70]:
from functools import partial
import pyproj
from pyspark import SparkContext
from geopyspark.geotrellis.color import ColorMap, get_colors_from_matplotlib
from geopyspark.geotrellis.tms import TMS
from geonotebook.wrappers import TMSRasterData
from shapely.geometry import MultiPolygon, shape
from shapely.ops import transform
import geopyspark as gps
import geopandas as gpd
import fiona

In [52]:
from overlay.data import data, mask

### config spark

In [5]:
conf=gps.geopyspark_conf(appName="ClimateOverlay")
conf.set('spark.master.memory', '9500M')
sc = SparkContext(conf = conf)

# subsetting inundation raster

In [6]:
# polygon mask
msas = gpd.read_file('/home/hadoop/notebooks/data/Boundaries/tl_2015_us_cbsa.shp')
nola = msas[msas['NAME'].str.contains('New Orleans', case = False)]

In [7]:
# clip raster
fulltiff = '/home/hadoop/notebooks/data/US_SLOSH_MOM_Inundation_v2/US_Category5_MOM_Inundation_HighTide.tif'
mask.clipRasterFromGDF(fulltiff, nola, '/tmp/nola.tif')

# visualizing inundation raster

In [8]:
inund = gps.geotiff.get(
    layer_type='spatial', 
    uri='/tmp/nola.tif')

In [9]:
inund_tiled = inund.tile_to_layout(
    layout=gps.GlobalLayout(), 
    target_crs=3857)
inund_pyramided = inund_tiled.pyramid().cache()

In [11]:
inund_histo         = inund_pyramided.get_histogram()
inund_colors        = get_colors_from_matplotlib('viridis', 100)
inund_color_map     = gps.ColorMap.from_histogram(inund_histo, inund_colors)

In [12]:
inund_tms = gps.TMS.build(inund_pyramided, inund_color_map)

In [81]:
M.set_center(-95.9617495,29.8159954, 9)

In [83]:
M.add_layer(TMSRasterData(inund_tms))

### sea level rise

In [18]:
# download and unzip sea level rise polygons
u = "https://coast.noaa.gov/htdata/Inundation/SLR/SLRdata/LA/LA_3_slr_data_dist.zip"
f = "/tmp/slr.zip"
data.sea_level_rise(u, f)

In [84]:
help(fiona.open)

Help on function open in module fiona:

open(path, mode='r', driver=None, schema=None, crs=None, encoding=None, layer=None, vfs=None, enabled_drivers=None, crs_wkt=None)
    Open file at ``path`` in ``mode`` "r" (read), "a" (append), or
    "w" (write) and return a ``Collection`` object.
    
    In write mode, a driver name such as "ESRI Shapefile" or "GPX" (see
    OGR docs or ``ogr2ogr --help`` on the command line) and a schema
    mapping such as:
    
      {'geometry': 'Point',
       'properties': [('class', 'int'), ('label', 'str'),
                      ('value', 'float')]}
    
    must be provided. If a particular ordering of properties ("fields"
    in GIS parlance) in the written file is desired, a list of (key,
    value) pairs as above or an ordered dict is required. If no ordering
    is needed, a standard dict will suffice.
    
    A coordinate reference system for collections in write mode can be
    defined by the ``crs`` parameter. It takes Proj4 style mappings lik

In [49]:
# for i, name in enumerate (fiona.listlayers(sourceDatabaseFile)):
#     with fiona.open(sourceDatabaseFile, layer=1) as gdb:
#         print("Current layer's [", name, "] feature count is: ", len(gdb))

In [40]:
t = fiona.open(sourceDatabaseFile, layer=1)

In [45]:
gdb = '/tmp/slr/LA_3_slr_final_dist_polys.gdb'
with fiona.open(gdb, layer=1) as source:
    slr_crs = source.crs['init']
    slr = [MultiPolygon(shape(poly['geometry'])) for poly in source]

In [71]:
def create_partial_reprojection_func(crs):
    return partial(pyproj.transform,
                   pyproj.Proj(init=crs),
                   pyproj.Proj(init='epsg:3857'))

reprojected_slr = [transform(create_partial_reprojection_func(slr_crs), s) for s in slr]

In [72]:
# Rasterize slr polygons
rasterize_options = gps.RasterizerOptions(
    includePartial=True, sampleType='PixelIsArea')

slr_raster = gps.rasterize(
    geoms=reprojected_slr,
    crs="EPSG:3857",
    zoom=12,
    fill_value=1,
    cell_type=gps.CellType.FLOAT32,
    options=rasterize_options,
    num_partitions=50)

slr_pyramid = slr_raster.pyramid()

In [73]:
slr_histo         = slr_pyramid.get_histogram()
slr_colors        = get_colors_from_matplotlib('viridis', 100)
slr_color_map     = gps.ColorMap.from_histogram(slr_histo, slr_colors)

In [74]:
slr_tms = gps.TMS.build(slr_pyramid, slr_color_map)